In [ ]:
%matplotlib notebook

In [ ]:
%gui qt

## Peer Review Demo

This demo uses a laptop web camera as an **Ophyd** device to illustrate how **typhon** and a **Jupyter Notebook** can work in tandem to help guide an operator through a specific process

First, let us load the laptop and create and automatically create a working display:

In [ ]:
from laptop import lp

The laptop should be ready to use. Simply calling `trigger` will grab a new image and push it through the facial recognition pipeline

In [ ]:
status = lp.trigger()

Let's take a look at the image in the notebook real quick using `matplotlib`

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(lp.image.get())

## Display Setup
**Typhon** should quickly be able to autogenerate a screen for us. We just have to add it to the suite

In [ ]:
import typhon
suite = typhon.TyphonSuite()
suite.show()

In [ ]:
suite.add_device(lp)
suite.show_subdisplay(lp)

Not bad, we can do better. Let us load some of the nice templates we made before. In practice, the rules of which devices use which screens will be kept in a common file so this step will be unnecessary. We can also switch to the darker style that ships with **Typhon**

In [ ]:
import os.path

embedded = os.path.abspath('embedded_screen.ui')
detailed = os.path.abspath('detailed_screen.ui')
suite.setStyleSheet("""\
TyphonDeviceDisplay[device_class='laptop.Laptop']
    {{qproperty-force_template: '{}';}}

TyphonDeviceDisplay[device_class='laptop.Laptop'][display_type='1']
    {{qproperty-force_template: '{}';}}
""".format(embedded, detailed))

In [ ]:
typhon.use_stylesheet(dark=True)

## Automated Procedures

Besides giving us a nice way to inspect the structure of devices, **Ophyd** also gives a uniform interface to **bluesky**. In other words, since we have implemented the **trigger** function. **bluesky** can use our laptop as a detector in any of its builtin scans. Let's setup a **RunEngine** and check it out

In [ ]:
import bluesky.plans as bp
from bluesky import RunEngine
from bluesky.callbacks import LivePlot
from bluesky.utils import install_qt_kicker, install_nb_kicker

install_qt_kicker(update_rate=0.01)
install_nb_kicker()

In [ ]:
RE = RunEngine()
face_plot = LivePlot('teddy_laptop_faces')

We can contract the ROI that our webcam uses by an arbitrary value either through the command line or through the GUI

In [ ]:
lp.h.put(200)

Now we are going to execute a very simple scan where we move our ROI vertically downwards. We want to use the laptop as our only detector, and then we can use the **y** configuration attribute to scan from two points with a specified number of steps. We also pass in our plotting callback to give some in-situ feedback

In [ ]:
RE(bp.scan([lp], lp.y, 100, 500, 10), (face_plot))

Now we may have a routine that requires some action from the operator. In these cases we often know the end condition we are looking for, just not how to get there. For instance, the scan below will continue taking pictures until we see a U.S President.

In [ ]:
president_plot = LivePlot('teddy_laptop_presidents')

In [ ]:
from plans import wait_for_value

In [ ]:
RE(wait_for_value([lp], 'teddy_laptop_presidents', 1, 0), (face_plot, president_plot))